# Prefetch example

In [1]:
import sys
import feasst as fst

def mc(target_acceptance=0.2,
       tunable_param=0.185,
       density=0.85,
       cutoff=3,
       temperature=0.88,
       trials_per=int(1e6),
       ):
    box_length = 2*cutoff
    file_app = "_a" + str(target_acceptance) + "_r" + str(cutoff)
    num_particles=int(density*box_length**3)
    print('num_particles', num_particles)
    print('target_acceptance', target_acceptance)
    print('tunable_param', tunable_param)

    monte_carlo = fst.Prefetch(fst.args({"trials_per_check": "10000000"}))
    monte_carlo.activate_prefetch(False)
    monte_carlo.add(fst.MakeConfiguration(fst.args({"cubic_side_length": str(box_length),
                                       "particle_type0": fst.install_dir() + "/particle/lj.fstprt"})))
    monte_carlo.add(fst.MakePotential(fst.MakeLennardJones()))
    monte_carlo.add(fst.MakePotential(fst.MakeLongRangeCorrections()))
    monte_carlo.set(fst.MakeThermoParams(fst.args({
        "beta": str(1./temperature),
        "chemical_potential": "1."})))
    monte_carlo.set(fst.MakeMetropolis())
    monte_carlo.add(fst.MakeTrialTranslate(fst.args({
        "weight": "1.",
        "tunable_param": str(tunable_param),
        "tunable_target_acceptance": str(target_acceptance),
        "tunable_percent_change": "0.01"})))
    monte_carlo.add(fst.MakeTrialAdd(fst.args({"particle_type": "0"})))
    monte_carlo.run(fst.MakeRun(fst.args({"until_num_particles": str(num_particles)})))
    monte_carlo.run(fst.MakeRemoveTrial(fst.args({"name": "TrialAdd"})))
    monte_carlo.add(fst.MakeLogAndMovie(fst.args({
        "trials_per" : str(trials_per),
        "output_file": "movie"+file_app,
        "clear_file": "true"})))
    monte_carlo.add(fst.MakeCheckEnergy(fst.args({"trials_per" : str(trials_per)})))
    monte_carlo.add(fst.MakeTune())
    monte_carlo.attempt(int(1e6)) #equilibrate
    monte_carlo.activate_prefetch(True)
    monte_carlo.add(fst.MakeMeanSquaredDisplacement(fst.args({
        "trials_per_update": "10000",
        "updates_per_origin": "1000",
        "output_file": "msd" + file_app + ".txt",
        "trials_per_write": str(int(1e5))})))
    monte_carlo.add(fst.MakeCPUTime(fst.args({
        "trials_per_update": str(trials_per),
        "trials_per_write": str(trials_per),
        "output_file": "cpu" + file_app + ".txt"})))
    monte_carlo.attempt(int(1e6))

mc()

num_particles 183
target_acceptance 0.2
tunable_param 0.185


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!